In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer

In [3]:
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph

In [4]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [5]:
parent_dir = os.path.abspath('..')
test_file = '/data/20news/20news_test_tsne.pkl'
with open(parent_dir+test_file, 'rb') as f:
    [inputs, labels, X_emb] = pickle.load(f)

In [6]:
print(inputs.shape)
print(len(labels))
print(X_emb.shape)

(2082, 20)
2082
(2082, 2)


In [7]:
G = DataEmbeddingGraph(inputs, [])

In [8]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = 20 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1
net_parameters['n_units_1'] = net_parameters['n_units_2'] = net_parameters['H']

In [9]:
# Graph net
net = GraphConvNet2(net_parameters)
if torch.cuda.is_available(): net.cuda()
root = 'results/20news_preprocessed_tsne3/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

In [10]:
# Net
if torch.cuda.is_available():   
    y_pred = net.forward(G).cpu().detach().numpy()
else:    
    y_pred = net.forward(G).detach().numpy()

In [ ]:
train_file = '/data/20news/20news_train_tsne.pkl'
with open(parent_dir+train_file, 'rb') as f:
    [inputs1, labels1, X_emb1] = pickle.load(f)

In [ ]:
data_file = '/data/20news/20news_metadata.pkl'
with open(parent_dir+data_file, 'rb') as f:
    [lda_model, topic_summaries, target_names] = pickle.load(f)

In [11]:
import bokeh.plotting as bp
from bokeh.plotting import save, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook

n_top_words = 5 # number of keywords we show

# 20 colors
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])
output_notebook()

Loading BokehJS ...

In [12]:
lda_keys = []
for i in range(inputs.shape[0]):
    lda_keys +=  inputs[i].argmax(),
print(len(lda_keys))

2082


In [21]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [42]:
from bokeh.models import Legend

named_labels = [target_names[i] for i in labels]
plot_lda = bp.figure(plot_width=1000, plot_height=600,
                     tools="pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

data_dict = {'x': y_pred[:, 0], 
             'y': y_pred[:, 1],
             'color': colormap[labels], 
             'label': named_labels}

mySource = bp.ColumnDataSource(data_dict)

plot_lda.circle(x='x', y='y', color='color', legend='label', source=mySource)
plot_lda.legend.location = (0, 70)
new_legend = plot_lda.legend[0]
plot_lda.legend[0].plot = None
plot_lda.add_layout(new_legend, 'right')
plot_lda.legend.label_text_font_size = '7pt'

# Select the centroid of each LDA cluster as the coordinates to plot crucial words
lda_labels = np.array(lda_keys)
topic_coord = np.zeros((inputs.shape[1], 2))
for i in range(inputs.shape[1]):
    mask = np.argwhere(lda_labels == i).flatten()
    topic_coord[i] = np.average(y_pred[mask], axis=0)
    
# plot crucial words
for i in range(inputs1.shape[1]):
      plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]], text_font_size='10px')
        
show(plot_lda)

In [16]:
train_lda_keys = []
for i in range(inputs1.shape[0]):
    train_lda_keys +=  inputs1[i].argmax(),

In [45]:
named_labels = [target_names[i] for i in labels1]

plot_lda = bp.figure(plot_width=1000, plot_height=600,
                     tools="pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

data_dict = {'x': X_emb1[:, 0], 
             'y': X_emb1[:, 1],
             'color': colormap[labels1], 
             'label': named_labels}

mySource = bp.ColumnDataSource(data_dict)

plot_lda.circle(x='x', y='y', color='color', legend='label', source=mySource)
plot_lda.legend.location = (0, 70)
new_legend = plot_lda.legend[0]
plot_lda.legend[0].plot = None
plot_lda.add_layout(new_legend, 'right')
plot_lda.legend.label_text_font_size = '7pt'

# Select the centroid of each LDA cluster as the coordinates to plot crucial words
train_lda_labels = np.array(train_lda_keys)
topic_coord = np.zeros((inputs1.shape[1], 2))
for i in range(inputs1.shape[1]):
    mask = np.argwhere(train_lda_labels == i).flatten()
    topic_coord[i] = np.average(X_emb1[mask], axis=0)
    
# plot crucial words
for i in range(inputs1.shape[1]):
      plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]], text_font_size='10px')

show(plot_lda)

In [53]:
from util.evaluation_metrics import nearest_neighbours_generalisation_accuracy
from sklearn.manifold.t_sne import trustworthiness

In [54]:
# Graph net
trust = trustworthiness(inputs, y_pred, n_neighbors=5)
one_nn = nearest_neighbours_generalisation_accuracy(y_pred, labels, 1)
five_nn = nearest_neighbours_generalisation_accuracy(y_pred, labels, 5)
print("Trust = {:.4f}, 1-NN = {:.4f}, 5-NN = {:.4f}".format(trust, one_nn, five_nn))

Trust = 0.9609, 1-NN = 0.3141, 5-NN = 0.3616


In [55]:
# tSNE
trust = trustworthiness(inputs, X_emb, n_neighbors=5)
one_nn = nearest_neighbours_generalisation_accuracy(X_emb, labels, 1)
five_nn = nearest_neighbours_generalisation_accuracy(X_emb, labels, 5)
print("Trust = {:.4f}, 1-NN = {:.4f}, 5-NN = {:.4f}".format(trust, one_nn, five_nn))

Trust = 0.9969, 1-NN = 0.3867, 5-NN = 0.4078
